# Bone Fracture Detection - Google Colab Setup

This notebook sets up and trains YOLO model for bone fracture detection with GPU support.

## Features:
- ✅ Free GPU access (T4, 16GB VRAM)
- ✅ Complete training pipeline
- ✅ Automatic model download
- ✅ Training visualization


In [1]:
# Install dependencies
!pip install ultralytics opencv-python opencv-contrib-python scikit-image scipy -q
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 -q

print("✅ Dependencies installed!")



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Dependencies installed!



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Check GPU/CPU
import torch
print("="*60)
print("DEVICE INFORMATION")
print("="*60)

if torch.cuda.is_available():
    print(f"✅ CUDA Available: True")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"VRAM: {vram_gb:.2f} GB")
    print("✅ GPU is ready for training!")
    recommended_model = 's' if vram_gb >= 8 else 'n'
    print(f"Recommended: YOLOv8{recommended_model}")
    device_type = "GPU"
else:
    print("ℹ️  CUDA Available: False")
    print("✅ CPU mode enabled - Training will work but will be slower")
    print("💡 Tip: For faster training, go to Runtime > Change runtime type > GPU (T4)")
    recommended_model = 'n'  # Use nano model for CPU
    device_type = "CPU"
print("="*60)


GPU INFORMATION
CUDA Available: False
⚠️  No GPU! Go to: Runtime > Change runtime type > GPU


## Step 2: Upload Dataset

**Option A**: Upload from your computer (click folder icon on left)
**Option B**: Mount Google Drive and copy dataset


In [ ]:
# Mount Google Drive (if dataset is in Drive)
# Note: This will only work in Google Colab, not in local Jupyter
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted!")
except ImportError:
    print("ℹ️  Running locally - Google Drive mount skipped")
    print("✅ Ready for dataset upload (use local paths)")

# Or upload dataset.zip manually and unzip:
# !unzip -q /content/archive.zip -d /content/bone_fracture_detection/data/


## Step 3: Training Configuration

Modify these settings as needed:


In [ ]:
# Training Configuration
DATA_YAML = "/content/bone_fracture_detection/data/archive/bone fracture detection.v4-v4.yolov8/data.yaml"

# Auto-adjust based on device
if torch.cuda.is_available():
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    if vram_gb >= 16:
        MODEL_SIZE = "m"  # medium
        BATCH = 32
    elif vram_gb >= 8:
        MODEL_SIZE = "s"  # small
        BATCH = 16
    else:
        MODEL_SIZE = "n"  # nano
        BATCH = 8
else:
    MODEL_SIZE = "n"  # nano for CPU
    BATCH = 4  # Smaller batch for CPU

EPOCHS = 10       # 5-10 epochs
IMGSZ = 640       # Image size

print(f"Configuration ({device_type}):")
print(f"  Model: YOLOv8{MODEL_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH} (adjusted for {device_type})")
print(f"  Image Size: {IMGSZ}")


## Step 4: Update Data YAML and Start Training


In [ ]:
from ultralytics import YOLO
import yaml
from pathlib import Path

# Update data.yaml paths
data_yaml_path = Path(DATA_YAML)
if data_yaml_path.exists():
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    base_dir = data_yaml_path.parent
    data_config['train'] = str(base_dir / 'train' / 'images')
    data_config['val'] = str(base_dir / 'valid' / 'images')
    data_config['test'] = str(base_dir / 'test' / 'images')
    
    updated_yaml = base_dir / 'data_updated.yaml'
    with open(updated_yaml, 'w') as f:
        yaml.dump(data_config, f)
    
    DATA_YAML = str(updated_yaml)
    print(f"✅ Data YAML updated: {updated_yaml}")
else:
    print(f"⚠️  Dataset not found. Please upload dataset first!")

# Start Training
print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)
print(f"This will take ~2-5 hours for {EPOCHS} epochs...")
print("="*80)

model = YOLO(f"yolov8{MODEL_SIZE}.pt")

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=0 if torch.cuda.is_available() else 'cpu',
    project="/content/bone_fracture_detection/yolo_training_results",
    name=f"yolov8{MODEL_SIZE}_bone_fracture",
    save=True,
    plots=True,
    verbose=True
)

print("\n✅ Training Complete!")


## Step 5: DenseNet-121 Fine-Tuning

Fine-tune DenseNet-121 for bone fracture classification with two-phase training.


In [ ]:
# Install additional dependencies for DenseNet training
%pip install scikit-learn pandas matplotlib seaborn tqdm -q
print("✅ Additional dependencies installed!")


In [ ]:
# Option 1: Clone from GitHub (Recommended)
# Uncomment the line below to clone your project
# !cd /content && git clone https://github.com/wareeshayyyyy/FCV_PROJECT.git bone_fracture_detection

# Option 2: Setup manually
import os
from pathlib import Path

base_dir = Path('/content/bone_fracture_detection')
if not (base_dir / 'run_complete_training.py').exists():
    # Create directory structure if not cloned
    base_dir.mkdir(exist_ok=True)
    os.makedirs(base_dir / 'src/bonefracture', exist_ok=True)
    os.makedirs(base_dir / 'checkpoints', exist_ok=True)
    os.makedirs(base_dir / 'training_results', exist_ok=True)
    
    print("✅ Directory structure created!")
    print("\n📁 Next steps:")
    print("1. Upload your dataset to:", base_dir / 'data/archive/')
    print("2. Upload run_complete_training.py to:", base_dir)
    print("3. Upload src/ folder to:", base_dir / 'src/')
    print("\nOr uncomment the git clone line above to clone from GitHub!")
else:
    print("✅ Project already cloned/uploaded!")
    print(f"📁 Project directory: {base_dir}")


## Step 6: Run DenseNet Fine-Tuning

This will run two-phase training:
- **Phase 1**: Train classifier with frozen backbone (10 epochs)
- **Phase 2**: Fine-tune all layers with differential LR (10 epochs)


In [ ]:
# DenseNet Fine-Tuning Configuration
import sys
from pathlib import Path

base_dir = Path('/content/bone_fracture_detection')
sys.path.insert(0, str(base_dir))

# Update dataset path for Colab (modify if your dataset is in a different location)
DATASET_ROOT = base_dir / "data/archive/bone fracture detection.v4-v4.yolov8"

# Update the path in run_complete_training.py if needed
if (base_dir / 'run_complete_training.py').exists():
    # Read and update the DATASET_ROOT in the script
    with open(base_dir / 'run_complete_training.py', 'r') as f:
        script_content = f.read()
    
    # Replace Windows path with Colab path
    script_content = script_content.replace(
        r"DATASET_ROOT = r'data\archive\bone fracture detection.v4-v4.yolov8'",
        f"DATASET_ROOT = r'{DATASET_ROOT}'"
    )
    
    with open(base_dir / 'run_complete_training.py', 'w') as f:
        f.write(script_content)
    
    print("✅ Updated dataset path in run_complete_training.py")

# Auto-adjust batch size for DenseNet based on device
if torch.cuda.is_available():
    densenet_batch = 16
else:
    densenet_batch = 4  # Smaller batch for CPU

print("="*80)
print("DENSENET-121 FINE-TUNING CONFIGURATION")
print("="*80)
print(f"Dataset: {DATASET_ROOT}")
print(f"Dataset exists: {Path(DATASET_ROOT).exists()}")
if torch.cuda.is_available():
    print(f"Device: GPU - {torch.cuda.get_device_name(0)}")
else:
    print(f"Device: CPU (Training will be slower but will work)")
print(f"Batch Size: {densenet_batch} (auto-adjusted for device)")
print(f"Phase 1 Epochs: 10 (Classifier training)")
print(f"Phase 2 Epochs: 10 (Full fine-tuning)")
print("="*80)

# Update batch size in the training script
if (base_dir / 'run_complete_training.py').exists():
    with open(base_dir / 'run_complete_training.py', 'r') as f:
        script_content = f.read()
    
    # Update batch size
    script_content = script_content.replace(
        "BATCH_SIZE = 16",
        f"BATCH_SIZE = {densenet_batch}"
    )
    
    with open(base_dir / 'run_complete_training.py', 'w') as f:
        f.write(script_content)
    
    print(f"✅ Updated batch size to {densenet_batch} in run_complete_training.py")


In [ ]:
# Run fine-tuning
# Make sure run_complete_training.py is uploaded and dataset path is correct

import subprocess
import os
import sys

os.chdir('/content/bone_fracture_detection')
sys.path.insert(0, '/content/bone_fracture_detection')

# Check if file exists
if os.path.exists('run_complete_training.py'):
    print("✅ Starting DenseNet fine-tuning...")
    print("This will take ~1-2 hours for 20 epochs total...")
    print("="*80)
    
    # Update dataset path in the script if needed
    # The script should use: DATASET_ROOT = '/content/bone_fracture_detection/data/archive/bone fracture detection.v4-v4.yolov8'
    
    # Run training (use exec to see real-time output)
    exec(open('run_complete_training.py').read())
else:
    print("⚠️  run_complete_training.py not found!")
    print("Please upload the file to /content/bone_fracture_detection/")
    print("\nOr clone from GitHub:")
    print("!cd /content && git clone https://github.com/wareeshayyyyy/FCV_PROJECT.git bone_fracture_detection")


In [ ]:
# Display training results and download models
from IPython.display import Image, display
from pathlib import Path
from google.colab import files

results_dir = Path('/content/bone_fracture_detection/training_results')
checkpoints_dir = Path('/content/bone_fracture_detection/checkpoints')

print("="*80)
print("TRAINING RESULTS")
print("="*80)

# Display training history plot
history_plot = results_dir / 'complete_training_history.png'
if history_plot.exists():
    display(Image(str(history_plot)))
    print("✅ Training history plot displayed!")
else:
    print("⚠️  Training history plot not found yet.")

# Download best models
models_to_download = [
    checkpoints_dir / 'best_model_phase_1.pth',
    checkpoints_dir / 'best_model_phase_2.pth',
    checkpoints_dir / 'final_model_complete.pth'
]

print("\n📥 Downloading models...")
for model_path in models_to_download:
    if model_path.exists():
        files.download(str(model_path))
        print(f"✅ Downloaded: {model_path.name}")
        
        # Also save to Drive (if mounted)
        try:
            from google.colab import drive
            import shutil
            drive_path = Path('/content/drive/MyDrive') / model_path.name
            if Path('/content/drive/MyDrive').exists():
                shutil.copy(model_path, drive_path)
                print(f"✅ Saved to Drive: {model_path.name}")
            else:
                print(f"ℹ️  Google Drive not mounted - skipping Drive save")
        except (ImportError, Exception):
            print(f"ℹ️  Google Drive not available - skipping Drive save")

# Download training results JSON
results_json = results_dir / 'complete_training_results.json'
if results_json.exists():
    files.download(str(results_json))
    print("✅ Training results JSON downloaded!")
    
    # Display results summary
    import json
    with open(results_json, 'r') as f:
        results = json.load(f)
    print("\n📊 Final Results Summary:")
    print(f"  Phase 1 Best Val Accuracy: {results.get('phase1_best_val_acc', 'N/A'):.4f}")
    print(f"  Phase 2 Best Val Accuracy: {results.get('phase2_best_val_acc', 'N/A'):.4f}")
    print(f"  Final Test Accuracy: {results.get('final_test_acc', 'N/A'):.4f}")
else:
    print("⚠️  Training results JSON not found yet.")


In [ ]:
# Display training results
from IPython.display import Image, display

results_dir = f"/content/bone_fracture_detection/yolo_training_results/yolov8{MODEL_SIZE}_bone_fracture"
best_model = f"{results_dir}/weights/best.pt"

try:
    display(Image(f"{results_dir}/results.png"))
    display(Image(f"{results_dir}/confusion_matrix.png"))
    print(f"✅ Best model: {best_model}")
except Exception as e:
    print(f"Results available in: {results_dir}")

# Download model
from google.colab import files
if Path(best_model).exists():
    files.download(best_model)
    print("✅ Model downloaded!")
    
    # Also save to Drive
    !cp {best_model} /content/drive/MyDrive/
    print("✅ Model saved to Google Drive!")
